In [ ]:
# Importando as bibliotecas necessárias
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import json
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Carregando e tratando os dataframes

### Vagas

In [ ]:
with open('vagas.json') as f:
    data_jobs = json.load(f)

df_jobs = pd.DataFrame.from_dict(data_jobs, orient='index')
df_jobs.index.name = 'vaga_id'
df_jobs.reset_index(inplace=True)

# Expandir colunas aninhadas
df_jobs_infos = pd.json_normalize(df_jobs['informacoes_basicas'])
df_jobs_perfil = pd.json_normalize(df_jobs['perfil_vaga'])
df_jobs_beneficios = pd.json_normalize(df_jobs['beneficios'])

# Concatenar
df_jobs_final = pd.concat([df_jobs[['vaga_id']], df_jobs_infos, df_jobs_perfil, df_jobs_beneficios], axis=1)


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
df_jobs_final.head()

,vaga_id,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,tipo_contratacao,prazo_contratacao,objetivo_vaga,prioridade_vaga,origem_vaga,superior_imediato,nome,telefone,data_inicial,data_final,nome_substituto,pais,estado,cidade,bairro,regiao,local_trabalho,vaga_especifica_para_pcd,faixa_etaria,horario_trabalho,nivel profissional,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2
0,5185,04-05-2021,00-00-0000,Operation Lead -,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Avançado,Fluente,,TI - Sistemas e Ferramentas-,Operations Lead\n\nRoles & Responsibilities:\n...,Required Skills:\n• Prior experience in Cloud ...,100% Remoto Período – entre 5 – 6 meses,,Nenhum -,NaN,-,R$,
1,5184,04-05-2021,00-00-0000,Consultor PP/QM Sênior,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Yasmin da Rosa,CLT Full,,Contratação,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Fluente,Nenhum,,TI - Desenvolvimento/Programação-,Consultor PP/QM Sr.\n\n• Consultor PP/QM Sênio...,• Consultor PP/QM Sênior com experiencia em pr...,• Início: Imediato • Fim: Jan/22,,Nenhum -,NaN,-,R$,
2,5183,04-05-2021,00-00-0000,ANALISTA PL/JR C/ SQL,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Albuquerque,CLT Full,,RFP,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Nenhum,Intermediário,,TI - Sistemas e Ferramentas-,Descrição – Atividades:\n\no Monitoramento das...,Requisitos mandatórios:\n\no Conhecimentos Téc...,Localização: Remoto Perfil: Analista Pleno ou ...,,Nenhum -,NaN,-,R$,
3,5182,04-05-2021,18-05-2021,Technical Architect - 11894809,Não,Nelson-Page,Dr. Raul Monteiro,Decision São Paulo,Cecília Freitas,Clara Rios,"PJ/Autônomo, CLT Full",Determinado,Contratação,Alta: Alta complexidade 3 a 5 dias,,Superior Imediato:,NaN,NaN,18-05-2021,17-01-2022,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Analista,Ensino Superior Completo,Básico,Básico,,TI - Projetos-,Descrição/Comentário: Architecture Frameworks ...,Descrição/Comentário: Architecture Frameworks ...,Budgeted Rate - indicate currency and type (ho...,Não,Notebook padrão -,NaN,- p/ mês (168h),fechado,
4,5181,04-05-2021,00-00-0000,Consultor SAP AUTHORIZATION (BCA) -Pleno / Sênior,Não,Mann and Sons,Cauê Fogaça,Decision São Paulo,Maria Laura Nogueira,Srta. Bella Ferreira,CLT Full,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,São Paulo,São Paulo,,,2000,Não,De: Até:,,Sênior,Ensino Superior Completo,Intermediário,Nenhum,,TI - SAP-,Experiência como Consultor SAP AUTHORIZATION (...,Experiência como Consultor SAP AUTHORIZATION (...,contratação CLT full pela Decision locação rem...,Sim,Nenhum -,NaN,-,R$,


In [ ]:
df_jobs_final.columns

Index(['vaga_id', 'data_requicisao', 'limite_esperado_para_contratacao',
       'titulo_vaga', 'vaga_sap', 'cliente', 'solicitante_cliente',
       'empresa_divisao', 'requisitante', 'analista_responsavel',
       'tipo_contratacao', 'prazo_contratacao', 'objetivo_vaga',
       'prioridade_vaga', 'origem_vaga', 'superior_imediato', 'nome',
       'telefone', 'data_inicial', 'data_final', 'nome_substituto', 'pais',
       'estado', 'cidade', 'bairro', 'regiao', 'local_trabalho',
       'vaga_especifica_para_pcd', 'faixa_etaria', 'horario_trabalho',
       'nivel profissional', 'nivel_academico', 'nivel_ingles',
       'nivel_espanhol', 'outro_idioma', 'areas_atuacao',
       'principais_atividades', 'competencia_tecnicas_e_comportamentais',
       'demais_observacoes', 'viagens_requeridas', 'equipamentos_necessarios',
       'habilidades_comportamentais_necessarias', 'valor_venda',
       'valor_compra_1', 'valor_compra_2'],
      dtype='object')

In [ ]:
df_jobs_final.shape

(14081, 45)

In [ ]:
# Criando uma coluna concatenada com o local da vaga
df_jobs_final['local_vaga'] = df_jobs_final['cidade'].str.strip() + ', ' + df_jobs_final['estado'].str.strip()

### Candidatos

In [ ]:
with open('applicants.json') as f:
    data_applicants = json.load(f)

df_applicants = pd.DataFrame.from_dict(data_applicants, orient='index')
df_applicants.index.name = 'codigo_candidato'
df_applicants.reset_index(inplace=True)

# Expandindo colunas
df_infos = pd.json_normalize(df_applicants['infos_basicas'])
df_pessoais = pd.json_normalize(df_applicants['informacoes_pessoais'])
df_profissionais = pd.json_normalize(df_applicants['informacoes_profissionais'])
df_formacao = pd.json_normalize(df_applicants['formacao_e_idiomas'])
df_cargo_atual = pd.json_normalize(df_applicants['cargo_atual'])
df_cv = pd.json_normalize(df_applicants['cv_pt'])

# Concatenando
df_applicants_final = pd.concat([
    df_applicants[['codigo_candidato']],
    df_infos, df_pessoais, df_profissionais, df_formacao, df_cargo_atual, df_cv
], axis=1)

In [ ]:
df_applicants_final.head()

,codigo_candidato,telefone_recado,telefone,objetivo_profissional,data_criacao,inserido_por,email,local,sabendo_de_nos_por,data_atualizacao,codigo_profissional,nome,data_aceite,nome,cpf,fonte_indicacao,email,email_secundario,data_nascimento,telefone_celular,telefone_recado,sexo,estado_civil,pcd,endereco,skype,url_linkedin,facebook,download_cv,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,outras_certificacoes,remuneracao,nivel_profissional,qualificacoes,experiencias,nivel_academico,nivel_ingles,nivel_espanhol,outro_idioma,instituicao_ensino_superior,cursos,ano_conclusao,outro_curso,id_ibrati,email_corporativo,cargo_atual,projeto_atual,cliente,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,email_superior_imediato
0,31000,,(11) 97048-2708,,10-11-2021 07:29:49,Luna Correia,carolina_aparecida@gmail.com,,,10-11-2021 07:29:49,31000,Carolina Aparecida,Cadastro anterior ao registro de aceite,Carolina Aparecida,,:,carolina_aparecida@gmail.com,,0000-00-00,(11) 97048-2708,,,,,,,,,NaN,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,31001,,(11) 93723-4396,Analista Administrativo,10-11-2021 08:56:16,Laura Pacheco,eduardo_rios@hotmail.com,"São Paulo, São Paulo",Outros,11-11-2021 11:10:31,31001,Eduardo Rios,Cadastro anterior ao registro de aceite,Eduardo Rios,,Outros: Contato do RH,eduardo_rios@hotmail.com,,28-12-1994,(11) 93723-4396,,Feminino,Solteiro,Não,são paulo,,,,NaN,Analista Administrativo,Administrativa,,,,1900,,NaN,NaN,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,31002,,(11) 92399-9824,Administrativo | Financeiro,10-11-2021 09:01:00,Laura Pacheco,pedro_henrique_carvalho@gmail.com,"São Paulo, São Paulo",Anúncio,10-11-2021 11:42:36,31002,Pedro Henrique Carvalho,Cadastro anterior ao registro de aceite,Pedro Henrique Carvalho,,Anúncio:,pedro_henrique_carvalho@gmail.com,,12-12-1988,(11) 92399-9824,,Feminino,Solteiro,Não,são paulo,,,,NaN,Administrativo | Financeiro,Administrativa,,"MS [77-418] MOS: Microsoft Office Word 2013, M...",,"2.500,00",,NaN,NaN,Ensino Superior Completo,Intermediário,Básico,Português - Fluente,,Administração de Empresas,2012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,31003,,(11) 98100-1727,Área administrativa,10-11-2021 09:08:13,Laura Pacheco,thiago_barbosa@hotmail.com,"São Paulo, São Paulo",Site de Empregos,10-11-2021 16:04:51,31003,Thiago Barbosa,Cadastro anterior ao registro de aceite,Thiago Barbosa,,Site de Empregos: Infojobs,thiago_barbosa@hotmail.com,,08-05-1992,(11) 98100-1727,,Feminino,Casado,Não,são paulo,,,,NaN,Área administrativa,Administrativa,,,,"1100,00",,NaN,NaN,Ensino Superior Incompleto,Nenhum,Nenhum,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,31004,,(11) 92517-2678,,10-11-2021 09:18:46,Maria Clara Pires,diogo_das_neves@hotmail.com,,,10-11-2021 09:22:03,31004,Diogo das Neves,Cadastro anterior ao registro de aceite,Diogo das Neves,,:,diogo_das_neves@hotmail.com,,31-12-1969,(11) 92517-2678,,,,,,,,,NaN,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
#print("Candidato (local):")
#print(df_final['local'].dropna().unique())

In [ ]:
df_applicants_final.columns

Index(['codigo_candidato', 'telefone_recado', 'telefone',
       'objetivo_profissional', 'data_criacao', 'inserido_por', 'email',
       'local', 'sabendo_de_nos_por', 'data_atualizacao',
       'codigo_profissional', 'nome', 'data_aceite', 'nome', 'cpf',
       'fonte_indicacao', 'email', 'email_secundario', 'data_nascimento',
       'telefone_celular', 'telefone_recado', 'sexo', 'estado_civil', 'pcd',
       'endereco', 'skype', 'url_linkedin', 'facebook', 'download_cv',
       'titulo_profissional', 'area_atuacao', 'conhecimentos_tecnicos',
       'certificacoes', 'outras_certificacoes', 'remuneracao',
       'nivel_profissional', 'qualificacoes', 'experiencias',
       'nivel_academico', 'nivel_ingles', 'nivel_espanhol', 'outro_idioma',
       'instituicao_ensino_superior', 'cursos', 'ano_conclusao', 'outro_curso',
       'id_ibrati', 'email_corporativo', 'cargo_atual', 'projeto_atual',
       'cliente', 'unidade', 'data_admissao', 'data_ultima_promocao',
       'nome_superior_ime

In [ ]:
df_applicants_final.shape

(42482, 56)

### Perspectivas

In [ ]:
with open('prospects.json', 'r') as f:
    data = json.load(f)

df = pd.DataFrame.from_dict(data, orient='index')
df.index.name = 'vaga_id'
df.reset_index(inplace=True)

# Explodindo a lista de prospects
df_exploded = df.explode('prospects').reset_index(drop=True)

# Expandindo os dicionários em colunas
df_prospects_final = pd.concat([
    df_exploded.drop(columns=['prospects']),
    pd.json_normalize(df_exploded['prospects'])
], axis=1)

df_prospects_final.head()

,vaga_id,titulo,modalidade,nome,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano
3,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Alexia Barbosa,25360,Encaminhado ao Requisitante,17-03-2021,17-03-2021,,Juliana Cassiano
4,4532,,,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_prospects_final.columns

Index(['vaga_id', 'titulo', 'modalidade', 'nome', 'codigo',
       'situacao_candidado', 'data_candidatura', 'ultima_atualizacao',
       'comentario', 'recrutador'],
      dtype='object')

In [ ]:
df_prospects_final.shape

(56702, 10)

### Tratando os dados

In [ ]:
# Cruzando as tabelas
df_final = df_prospects_final \
    .merge(df_applicants_final, left_on='codigo', right_on='codigo_candidato', how='left') \
    .merge(df_jobs_final, on='vaga_id', how='left')


In [ ]:
# Removendo colunas duplicadas
df_final = df_final.loc[:, ~df_final.columns.duplicated()]

In [ ]:
# Removendo algumas colunas aparentemente irrelevantes
colunas_descartar = [
    'cpf', 'telefone', 'telefone_recado', 'telefone_celular',
    'facebook', 'skype', 'url_linkedin', 'email', 'email_secundario',
    'email_superior_imediato', 'email_corporativo', 'download_cv'
]
df_final = df_final.drop(columns=[col for col in colunas_descartar if col in df_final.columns])

In [ ]:
# Verificando as colunas com maior/menor percentual de nulos, a fim de fazer melhores escolhas em relação aos campos utilizados no modelo.
df_final.isnull().mean().sort_values(ascending=False)


,0
outro_curso,0.984551
cliente_x,0.983634
id_ibrati,0.983634
projeto_atual,0.983634
cargo_atual,0.983634
...,...
demais_observacoes,0.002892
local_vaga,0.002892
modalidade,0.000000
vaga_id,0.000000


In [ ]:
df_final.head()

,vaga_id,titulo,modalidade,nome_x,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,codigo_candidato,telefone_x,objetivo_profissional,data_criacao,inserido_por,local,sabendo_de_nos_por,data_atualizacao,codigo_profissional,nome_y,data_aceite,fonte_indicacao,data_nascimento,sexo,estado_civil,pcd,endereco,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,outras_certificacoes,remuneracao,nivel_profissional,qualificacoes,experiencias,nivel_academico_x,nivel_ingles_x,nivel_espanhol_x,outro_idioma_x,instituicao_ensino_superior,cursos,ano_conclusao,outro_curso,id_ibrati,cargo_atual,projeto_atual,cliente_x,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente_y,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,tipo_contratacao,prazo_contratacao,objetivo_vaga,prioridade_vaga,origem_vaga,superior_imediato,nome,telefone_y,data_inicial,data_final,nome_substituto,pais,estado,cidade,bairro,regiao,local_trabalho,vaga_especifica_para_pcd,faixa_etaria,horario_trabalho,nivel profissional,nivel_academico_y,nivel_ingles_y,nivel_espanhol_y,outro_idioma_y,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2,local_vaga
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,25632,(21) 93485-6494,,25-03-2021 13:36:19,Ana Lívia Moreira,,,25-03-2021 13:36:19,25632,José Vieira,Cadastro anterior ao registro de aceite,:,0000-00-00,,,,,,,,,,,,NaN,NaN,,,,-,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Contratação PJ Projeto pontual de 2 a 3 meses ...,,Nenhum -,NaN,-,R$,,"Rio de Janeiro, Rio de Janeiro"
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,25529,(21) 94261-6276,ANALISTA DE REDES E TELEPROCESSAMENTO/INFRAEST...,22-03-2021 22:48:58,Ana Lívia Moreira,"Rio de Janeiro, Rio de Janeiro",Site de Empregos,22-03-2021 23:36:28,25529,Srta. Isabela Cavalcante,Cadastro anterior ao registro de aceite,Site de Empregos: Linkdin,25-10-1973,Masculino,Solteiro,Não,rio de janeiro,ANALISTA DE REDES E TELEPROCESSAMENTO/INFRAEST...,"TI - Governança, TI - Infraestrutura, TI - Pro...",,,,"R$ 5600,00 / mês / CLT",,NaN,NaN,Ensino Superior Completo,Intermediário,Básico,-,,,2020,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,NaN,NaN,NaN,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Contratação PJ Projeto pontual de 2 a 3 meses ...,,Nenhum -,NaN,-,R$,,"Rio de Janeiro, Rio de Janeiro"
2,4531,2021-2607395-PeopleSoft Application Engine-Dom...,,Sra. Yasmin Fernandes,25364,Contratado pela Decision,17-03-2021,12-04-2021,Data de Inicio: 12/04/2021,Juliana Cassiano,25364,(21) 96332-6963,Liderança / Desenvolvimento,17-03-2021 09:41:10,Juliana Cassiano,"São Paulo, São Paulo",Site de Empregos,12-04-2021 12:15:57,25364,Sra. Yasmin Fernandes,Cadast

In [ ]:
df_final = df_final.fillna('Não informado')


In [ ]:
df_final.head()

,vaga_id,titulo,modalidade,nome_x,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,codigo_candidato,telefone_x,objetivo_profissional,data_criacao,inserido_por,local,sabendo_de_nos_por,data_atualizacao,codigo_profissional,nome_y,data_aceite,fonte_indicacao,data_nascimento,sexo,estado_civil,pcd,endereco,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,outras_certificacoes,remuneracao,nivel_profissional,qualificacoes,experiencias,nivel_academico_x,nivel_ingles_x,nivel_espanhol_x,outro_idioma_x,instituicao_ensino_superior,cursos,ano_conclusao,outro_curso,id_ibrati,cargo_atual,projeto_atual,cliente_x,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente_y,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,tipo_contratacao,prazo_contratacao,objetivo_vaga,prioridade_vaga,origem_vaga,superior_imediato,nome,telefone_y,data_inicial,data_final,nome_substituto,pais,estado,cidade,bairro,regiao,local_trabalho,vaga_especifica_para_pcd,faixa_etaria,horario_trabalho,nivel profissional,nivel_academico_y,nivel_ingles_y,nivel_espanhol_y,outro_idioma_y,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2,local_vaga
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,Encaminhado ao Requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,25632,(21) 93485-6494,,25-03-2021 13:36:19,Ana Lívia Moreira,,,25-03-2021 13:36:19,25632,José Vieira,Cadastro anterior ao registro de aceite,:,0000-00-00,,,,,,,,,,,,Não informado,Não informado,,,,-,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,Não informado,Não informado,Não informado,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Contratação PJ Projeto pontual de 2 a 3 meses ...,,Nenhum -,Não informado,-,R$,,"Rio de Janeiro, Rio de Janeiro"
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,Encaminhado ao Requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,25529,(21) 94261-6276,ANALISTA DE REDES E TELEPROCESSAMENTO/INFRAEST...,22-03-2021 22:48:58,Ana Lívia Moreira,"Rio de Janeiro, Rio de Janeiro",Site de Empregos,22-03-2021 23:36:28,25529,Srta. Isabela Cavalcante,Cadastro anterior ao registro de aceite,Site de Empregos: Linkdin,25-10-1973,Masculino,Solteiro,Não,rio de janeiro,ANALISTA DE REDES E TELEPROCESSAMENTO/INFRAEST...,"TI - Governança, TI - Infraestrutura, TI - Pro...",,,,"R$ 5600,00 / mês / CLT",,Não informado,Não informado,Ensino Superior Completo,Intermediário,Básico,-,,,2020,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,Não informado,Não informado,Não informado,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Contratação PJ Projeto pontual de 2 a 3 meses ...,,Nenhum -,Não informado,-,R$,,"Rio de Janeiro, Rio de Janeiro"
2,4531,2021-2607395-PeopleSo

### Criando a variável target

In [ ]:
df_final['situacao_candidado'].value_counts()

,count
situacao_candidado,
Prospect,20021
Encaminhado ao Requisitante,16122
Inscrito,3980
Não Aprovado pelo Cliente,3492
Não informado,2943
Contratado pela Decision,2758
Desistiu,2349
Não Aprovado pelo RH,1765
Não Aprovado pelo Requisitante,765


In [ ]:
pip install unidecode

In [ ]:
from unidecode import unidecode

In [ ]:
df_final['situacao_candidado'] = df_final['situacao_candidado'].str.lower().map(unidecode)


Vamos considerar apenas os contratados e os encaminhados para o cliente, já que são candidatos considerados bons o suficiente.

In [ ]:
df_final['foi_contratado_encaminhado'] = df_final['situacao_candidado'].str.lower().isin([
    'contratado pela decision', 'contratado como hunting', 'encaminhado ao requisitante', 'aprovado',
    'entrevista tecnica	','entrevista com cliente', 'documentação clt', 'documentação pj', 'proposta aceita', 'encaminhar proposta', 'em avaliação pelo rh'
]).astype(int)

In [ ]:
df_final['foi_contratado_encaminhado'].value_counts(normalize=True)

,proportion
foi_contratado_encaminhado,
0,0.651035
1,0.348965


In [ ]:
pd.set_option('display.max_columns', None)

In [ ]:
df_final.head()

,vaga_id,titulo,modalidade,nome_x,codigo,situacao_candidado,data_candidatura,ultima_atualizacao,comentario,recrutador,codigo_candidato,telefone_x,objetivo_profissional,data_criacao,inserido_por,local,sabendo_de_nos_por,data_atualizacao,codigo_profissional,nome_y,data_aceite,fonte_indicacao,data_nascimento,sexo,estado_civil,pcd,endereco,titulo_profissional,area_atuacao,conhecimentos_tecnicos,certificacoes,outras_certificacoes,remuneracao,nivel_profissional,qualificacoes,experiencias,nivel_academico_x,nivel_ingles_x,nivel_espanhol_x,outro_idioma_x,instituicao_ensino_superior,cursos,ano_conclusao,outro_curso,id_ibrati,cargo_atual,projeto_atual,cliente_x,unidade,data_admissao,data_ultima_promocao,nome_superior_imediato,data_requicisao,limite_esperado_para_contratacao,titulo_vaga,vaga_sap,cliente_y,solicitante_cliente,empresa_divisao,requisitante,analista_responsavel,tipo_contratacao,prazo_contratacao,objetivo_vaga,prioridade_vaga,origem_vaga,superior_imediato,nome,telefone_y,data_inicial,data_final,nome_substituto,pais,estado,cidade,bairro,regiao,local_trabalho,vaga_especifica_para_pcd,faixa_etaria,horario_trabalho,nivel profissional,nivel_academico_y,nivel_ingles_y,nivel_espanhol_y,outro_idioma_y,areas_atuacao,principais_atividades,competencia_tecnicas_e_comportamentais,demais_observacoes,viagens_requeridas,equipamentos_necessarios,habilidades_comportamentais_necessarias,valor_venda,valor_compra_1,valor_compra_2,local_vaga,foi_contratado_encaminhado
0,4530,CONSULTOR CONTROL M,,José Vieira,25632,encaminhado ao requisitante,25-03-2021,25-03-2021,"Encaminhado para - PJ R$ 72,00/hora",Ana Lívia Moreira,25632,(21) 93485-6494,,25-03-2021 13:36:19,Ana Lívia Moreira,,,25-03-2021 13:36:19,25632,José Vieira,Cadastro anterior ao registro de aceite,:,0000-00-00,,,,,,,,,,,,Não informado,Não informado,,,,-,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,Não informado,Não informado,Não informado,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Contratação PJ Projeto pontual de 2 a 3 meses ...,,Nenhum -,Não informado,-,R$,,"Rio de Janeiro, Rio de Janeiro",1
1,4530,CONSULTOR CONTROL M,,Srta. Isabela Cavalcante,25529,encaminhado ao requisitante,22-03-2021,23-03-2021,"encaminhado para - R$ 6.000,00 – CLT Full , n...",Ana Lívia Moreira,25529,(21) 94261-6276,ANALISTA DE REDES E TELEPROCESSAMENTO/INFRAEST...,22-03-2021 22:48:58,Ana Lívia Moreira,"Rio de Janeiro, Rio de Janeiro",Site de Empregos,22-03-2021 23:36:28,25529,Srta. Isabela Cavalcante,Cadastro anterior ao registro de aceite,Site de Empregos: Linkdin,25-10-1973,Masculino,Solteiro,Não,rio de janeiro,ANALISTA DE REDES E TELEPROCESSAMENTO/INFRAEST...,"TI - Governança, TI - Infraestrutura, TI - Pro...",,,,"R$ 5600,00 / mês / CLT",,Não informado,Não informado,Ensino Superior Completo,Intermediário,Básico,-,,,2020,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,Não informado,10-03-2021,00-00-0000,CONSULTOR CONTROL M,Não,"Morris, Moran and Dodson",Dra. Catarina Marques,Decision São Paulo,Maria Laura Nogueira,Ana Lívia Moreira,PJ/Autônomo,,,,,Superior Imediato:,,,Não informado,Não informado,Não informado,Brasil,Rio de Janeiro,Rio de Janeiro,,,2000,Não,De: Até:,,Pleno,Ensino Superior Completo,Nenhum,Nenhum,,TI - Desenvolvimento/Programação-,- Experiência comprovada em projetos de control-M,- Experiência comprovada em projetos de control-M,Contratação PJ Projeto pontual de 2 a 3 meses ...,,Nenhum -,Não informado,-,R$,,"Rio de Janeiro, Rio de Janeiro"

### Match de local entre candidato e vaga

Foram cruzados os dados de localização do candidato (campo local) com o campo concatenado de estado e cidade das vagas. Foi criado o campo match_regiao, que indica se o candidato está na mesma localidade da vaga. Seria interessante aumentar o escopo do match futuramente, considerando cidades próximas.

In [ ]:
def calcular_match_local(vaga_local, cand_local):
    if pd.isna(vaga_local) or pd.isna(cand_local):
        return 0
    return int(vaga_local.strip().lower() == cand_local.strip().lower())

df_final['match_local'] = df_final.apply(
    lambda row: calcular_match_local(row['local_vaga'], row['local']),
    axis=1
)


In [ ]:
df_final[
    df_final['match_local'] == 1
][[
    'local_vaga', 'local', 'match_local'
]].sample(10)

,local_vaga,local,match_local
41002,"São Paulo, São Paulo","São Paulo, São Paulo",1
20610,"São Paulo, São Paulo","São Paulo, São Paulo",1
24521,"São Paulo, São Paulo","São Paulo, São Paulo",1
45073,"São Paulo, São Paulo","São Paulo, São Paulo",1
45964,"São Paulo, São Paulo","São Paulo, São Paulo",1
24699,"São Paulo, São Paulo","São Paulo, São Paulo",1
42215,"São Paulo, São Paulo","São Paulo, São Paulo",1
40219,"São Paulo, São Paulo","São Paulo, São Paulo",1
18035,"Fortaleza, Ceará","Fortaleza, Ceará",1
7254,"São Paulo, São Paulo","São Paulo, São Paulo",1


### Match de nível de inglês

Segue a mesma lógica do match de senioridade. A partir de valores textuais padronizados (Ex: Básico, Intermediário, Avançado, Fluente), foi atribuído um ranking ordinal e calculado se o candidato atende o mínimo exigido. O resultado foi salvo como:

match_ingles (1 se atende),

delta_ingles (a diferença entre os níveis da vaga e do candidato).

In [ ]:
print("Candidato (x):")
print(df_final['nivel_ingles_x'].dropna().unique())

print("\nVaga (y):")
print(df_final['nivel_ingles_y'].dropna().unique())


Candidato (x):
['' 'Intermediário' 'Avançado' 'Não informado' 'Fluente' 'Básico' 'Nenhum']

Vaga (y):
['Nenhum' 'Técnico' 'Básico' 'Avançado' 'Fluente' 'Intermediário'
 'Não informado']


In [ ]:
df_final['nivel_ingles_y'].value_counts()

,count
nivel_ingles_y,
Básico,20041
Nenhum,13192
Avançado,10539
Fluente,6357
Intermediário,5612
Técnico,797
Não informado,164


In [ ]:
df_final['nivel_ingles_x'].value_counts()

,count
nivel_ingles_x,
,33685
Não informado,11607
Intermediário,3949
Avançado,3166
Básico,2775
Fluente,1190
Nenhum,330


In [ ]:
ordem_ingles = {
    'nenhum': 0,
    'básico': 1,
    'intermediário': 2,
    'técnico': 3,
    'avançado':4,
    'fluente': 5,
    'não informado': -1,
    '': -1
}

In [ ]:
def mapear_nivel_ingles(valor):
    if pd.isna(valor): return -1
    return ordem_ingles.get(valor.strip().lower(), -1)

df_final['ingles_candidato'] = df_final['nivel_ingles_x'].apply(mapear_nivel_ingles)
df_final['ingles_vaga'] = df_final['nivel_ingles_y'].apply(mapear_nivel_ingles)

df_final['delta_ingles'] = df_final['ingles_candidato'] - df_final['ingles_vaga']
df_final['match_ingles_aceitavel'] = (df_final['delta_ingles'] >= 0).astype(int)



In [ ]:
df_final[
    df_final['delta_ingles'] >= 0
][[
    'nivel_ingles_x', 'nivel_ingles_y', 'ingles_candidato', 'ingles_vaga', 'delta_ingles'
]].sample(10)


,nivel_ingles_x,nivel_ingles_y,ingles_candidato,ingles_vaga,delta_ingles
16616,Fluente,Técnico,5,3,2
46682,Intermediário,Básico,2,1,1
34255,Avançado,Básico,4,1,3
13149,Fluente,Básico,5,1,4
41338,Intermediário,Básico,2,1,1
18764,Básico,Básico,1,1,0
27360,Intermediário,Intermediário,2,2,0
15443,Avançado,Avançado,4,4,0
31698,Básico,Básico,1,1,0
52352,Básico,Básico,1,1,0


### Match de nível acadêmico

Foi criado um mapeamento ordinal do nível acadêmico (Ex: Ensino Médio = 1, Graduação = 2, Pós = 3, etc.). Com isso, comparamos os níveis exigidos na vaga e o nível do candidato, criando duas variáveis:

match_nivel_academico (valor binário: 1 se o candidato atende ou supera o requerido),

e delta_academico (diferença entre os níveis, útil para análises mais detalhadas).

In [ ]:
print(df_final['nivel_academico_x'].value_counts())
print(df_final['nivel_academico_y'].value_counts())

nivel_academico_x
                                 33146
Não informado                    11607
Ensino Superior Completo          5319
Pós Graduação Completo            3044
Ensino Superior Cursando          1256
Ensino Superior Incompleto         574
Pós Graduação Cursando             494
Pós Graduação Incompleto           301
Mestrado Completo                  284
Ensino Técnico Completo            186
Ensino Médio Completo              172
Mestrado Incompleto                117
Mestrado Cursando                   51
Ensino Fundamental Completo         47
Ensino Técnico Cursando             22
Doutorado Incompleto                21
Doutorado Cursando                  18
Doutorado Completo                  14
Ensino Técnico Incompleto            8
Ensino Médio Incompleto              6
Ensino Fundamental Incompleto        6
Ensino Fundamental Cursando          6
Ensino Médio Cursando                3
Name: count, dtype: int64
nivel_academico_y
Ensino Superior Completo       43540
Ensi

In [ ]:
ordem_academica = {
    'ensino fundamental incompleto': 0,
    'ensino fundamental cursando': 1,
    'ensino fundamental completo': 2,
    'ensino médio incompleto': 3,
    'ensino médio cursando': 4,
    'ensino médio completo': 5,
    'ensino técnico incompleto': 6,
    'ensino técnico cursando': 7,
    'ensino técnico completo': 8,
    'ensino superior incompleto': 9,
    'ensino superior cursando': 10,
    'ensino superior completo': 11,
    'pós graduacao incompleto': 12,
    'pós graduacao cursando': 13,
    'pós graduacao completo': 14,
    'mestrado cursando': 15,
    'mestrado incompleto': 16,
    'mestrado completo': 17,
    'doutorado cursando': 18,
    'doutorado incompleto': 19,
    'doutorado completo': 20,
    'não informado': -1,
    '': -1
}

In [ ]:
def mapear_nivel_academico(valor):
    if pd.isna(valor): return -1
    valor = valor.strip().lower()
    return ordem_academica.get(valor, -1)

df_final['academico_vaga'] = df_final['nivel_academico_y'].apply(mapear_nivel_academico)
df_final['academico_candidato'] = df_final['nivel_academico_x'].apply(mapear_nivel_academico)

df_final['delta_academico'] = df_final['academico_candidato'] - df_final['academico_vaga']
df_final['match_academico_aceitavel'] = (df_final['delta_academico'] >= 0).astype(int)

In [ ]:
df_final[
    df_final['delta_academico'] >= 0
][[
    'nivel_academico_x',
    'nivel_academico_y',
    'academico_candidato',
    'academico_vaga',
    'delta_academico'
]].sample(10)



,nivel_academico_x,nivel_academico_y,academico_candidato,academico_vaga,delta_academico
7072,Ensino Superior Completo,Ensino Superior Cursando,11,10,1
46004,,Não informado,-1,-1,0
31431,Ensino Superior Completo,Ensino Técnico Completo,11,8,3
6793,Ensino Superior Completo,Ensino Superior Completo,11,11,0
13220,Ensino Superior Completo,Ensino Superior Completo,11,11,0
5430,Ensino Superior Completo,Ensino Superior Completo,11,11,0
5593,Ensino Superior Completo,Ensino Superior Completo,11,11,0
24819,Ensino Superior Cursando,Ensino Médio Completo,10,5,5
39757,Ensino Superior Completo,Ensino Superior Completo,11,11,0
53459,Ensino Superior Completo,Ensino Superior Completo,11,11,0


### Match de senioridade

Aqui foram utilizados os campo de nível profissional (Ex: Júnior, Pleno, Sênior) do candidato e da vaga, e realizados dois cálculos:
O delta_senioridade calcula a diferença direta entre senioridade da vaga e do candidato para avaliar compatibilidades e excessos.



In [ ]:
print("Candidato (senioridade):")
print(df_final['nivel_profissional'].dropna().unique())

print("\nVaga (senioridade):")
print(df_final['nivel profissional'].dropna().unique())

Candidato (senioridade):
['' 'Não informado' 'Pleno' 'Sênior' 'Líder' 'Júnior' 'Analista'
 'Especialista' 'Estagiário' 'Técnico de Nível Médio' 'Gerente']

Vaga (senioridade):
['Pleno' 'Sênior' 'Analista' 'Júnior' 'Especialista' 'Não informado'
 'Líder' 'Assistente' 'Gerente' 'Supervisor' 'Coordenador' 'Aprendiz'
 'Auxiliar' 'Trainee' 'Técnico de Nível Médio']


In [ ]:
ordem_senioridade = {
    'aprendiz': 0,
    'auxiliar': 1,
    'estagiário': 2,
    'trainee': 3,
    'assistente': 4,
    'técnico de nível médio': 5,
    'júnior': 6,
    'analista': 7,
    'pleno': 8,
    'especialista': 9,
    'líder': 10,
    'supervisor': 11,
    'coordenador': 12,
    'sênior': 13,
    'gerente': 14,
    'não informado': -1,
    '': -1
}


delta = 0 → candidato no nível exato

delta > 0 → candidato mais sênior do que a vaga

delta < 0 → candidato menos sênior

In [ ]:
def mapear_senioridade(valor):
    if pd.isna(valor): return -1
    valor = valor.strip().lower()
    return ordem_senioridade.get(valor, -1)

df_final['senioridade_vaga'] = df_final['nivel profissional'].apply(mapear_senioridade)
df_final['senioridade_candidato'] = df_final['nivel_profissional'].apply(mapear_senioridade)

# Criando um delta de diferença de senioridade
df_final['delta_senioridade'] = df_final['senioridade_candidato'] - df_final['senioridade_vaga']

# Match direto ou aceitável
df_final['match_senioridade'] = (df_final['delta_senioridade'] == 0).astype(int)

df_final['match_senioridade_aceitavel'] = (df_final['delta_senioridade'] >= 0).astype(int)


In [ ]:
df_final[
    df_final['delta_senioridade'] >= 0
][[
    'nivel_profissional', 'nivel profissional', 'senioridade_candidato', 'senioridade_vaga','delta_senioridade'
]].sample(10)


,nivel_profissional,nivel profissional,senioridade_candidato,senioridade_vaga,delta_senioridade
12576,Sênior,Pleno,13,8,5
46331,Não informado,Não informado,-1,-1,0
24351,Sênior,Sênior,13,13,0
52568,Não informado,Não informado,-1,-1,0
44006,Não informado,Não informado,-1,-1,0
11111,Especialista,Pleno,9,8,1
45881,Não informado,Não informado,-1,-1,0
48657,Sênior,Sênior,13,13,0
14786,Especialista,Analista,9,7,2
24268,Sênior,Analista,13,7,6


### Match de nível técnico

Para medir a similaridade entre os requisitos da vaga e o perfil do candidato, foram aplicadas Técnicas de Processamento de Linguagem Natural (PLN). Especificamente, concatenados os campos de competências comportamentais e técnicas tanto do candidato quanto da vaga. Em seguida, utilizado o TF-IDF (Term Frequency-Inverse Document Frequency) para transformar esses textos em vetores numéricos, considerando a frequência relativa das palavras. Para medir o quão parecidos os textos são, utilizamos a similaridade do cosseno, atribuindo valores próximos de 1 para perfis mais alinhados. Esse valor foi salvo na coluna match_competencias_v3.

In [ ]:
def calcular_match_tecnico(vaga, candidato):
    if pd.isna(vaga) or pd.isna(candidato):
        return 0

    # Limpar e padronizar
    vaga = vaga.strip().lower()
    candidato = candidato.strip().lower()

    # Verificar campos vazios ou inúteis
    if vaga in ['', '.', 'não informado'] or candidato in ['', '.', 'não informado']:
        return 0

    # Separar por vírgula e comparar
    set_vaga = set(vaga.replace('\n', ' ').replace('.', '').split(','))
    set_cand = set(candidato.replace('\n', ' ').replace('.', '').split(','))
    intersecao = set_vaga & set_cand

    return len(intersecao) / len(set_vaga) if set_vaga else 0

df_final['match_competencias'] = df_final.apply(
    lambda row: calcular_match_tecnico(
        row['competencia_tecnicas_e_comportamentais'],
        row['conhecimentos_tecnicos']
    ), axis=1
)


In [ ]:
df_final.sort_values(by='match_competencias', ascending=False)[[
    'competencia_tecnicas_e_comportamentais',
    'conhecimentos_tecnicos',
    'match_competencias'
]].head(10)


,competencia_tecnicas_e_comportamentais,conhecimentos_tecnicos,match_competencias
32828,Java,"java, spring, spring boot, spring batch, sprin...",1.000000
9955,SAP SD,SAP SD,1.000000
11115,"Conhecimento em Cobol, Cics, Cobol IMS, DB2, V...",Mais de 30 anos de desenvolvimento de sistemas...,0.555556
11125,"Conhecimento em Cobol, Cics, Cobol IMS, DB2, V...",Como analista de sistemas tenho experiência no...,0.555556
5990,"Mainframe\n\nLINGUAGENS\n\nCOBOL, TSO, ROSCOE,...",Como analista de sistemas tenho experiência no...,0.545455
4480,Scrum Desejável\n\nDiferenciais:\n• Experiênci...,"C#, .Net MVC / Core, REST, Entity Framework, A...",0.500000
846,Desejável:\n\n•Configurações no API Management...,▪ Manipulação de bases de dados diferentes. ▪ ...,0.500000
34773,• Desejável:\n\nConhecimento em frameworks de ...,"Java EE, Spring MVC, Spring Boot, AngularJS, A...",0.500000
13888,"Conhecimento do ambiente Bradesco, nas tecnolo...","• Alta plataforma: Mainframe IBM OS390, VM, VS...",0.500000
13889,"Conhecimento do ambiente Bradesco, nas tecnolo...",• Profissional com experiência na área de info...,0.500000


In [ ]:
# Definir textos inválidos
textos_invalidos = ['', 'não informado', 'Não informado', '.', '-', 'nan']

# Filtrar onde tanto a vaga quanto o candidato têm informações válidas
df_final = df_final[
    (~df_final['competencia_tecnicas_e_comportamentais'].str.lower().isin(textos_invalidos)) &
    (~df_final['conhecimentos_tecnicos'].str.lower().isin(textos_invalidos))
]


In [ ]:
# Baixar stopwords
nltk.download('stopwords')
stopwords_pt = set(stopwords.words('portuguese'))

def limpar_texto(texto):
    if pd.isna(texto) or texto.lower() in ['não informado', '.', '']:
        return ''

    texto = texto.lower()
    texto = re.sub(r'\b(experiência|conhecimento|conhecimentos|desejável|desejaveis|profissional com|analista com|.*com\s+experiência)\b', '', texto)
    texto = re.sub(r'[^a-zA-Zçãáâàéêíóôõúü\s/-]', ' ', texto)
    texto = texto.replace('/', ' ').replace('-', ' ').replace('\n', ' ')
    tokens = [palavra for palavra in texto.split() if palavra not in stopwords_pt and len(palavra) > 2]
    return ' '.join(tokens)

# Aplicar ao df
df_final['comp_vaga_limpa'] = df_final['competencia_tecnicas_e_comportamentais'].apply(limpar_texto)
df_final['comp_cand_limpa'] = df_final['conhecimentos_tecnicos'].apply(limpar_texto)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Concatenar para vetorizar junto
corpus = pd.concat([df_final['comp_vaga_limpa'], df_final['comp_cand_limpa']], axis=0).fillna('')

vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(corpus)

# Separar novamente os vetores
n = len(df_final)
vaga_tfidf = tfidf_matrix[:n]
cand_tfidf = tfidf_matrix[n:]

# Similaridade de cosseno
similaridades = cosine_similarity(vaga_tfidf, cand_tfidf)

# Similaridade da diagonal (vaga-candidato pareados)
df_final['match_competencias_v3'] = np.diag(similaridades)


In [ ]:
df_final = df_final[
    (df_final['comp_vaga_limpa'] != '') &
    (df_final['comp_cand_limpa'] != '')
].copy()


In [ ]:
df_final.sort_values(by='match_competencias_v3', ascending=False)[[
    'competencia_tecnicas_e_comportamentais',
    'conhecimentos_tecnicos',
    'match_competencias_v3'
]].head(10)


,competencia_tecnicas_e_comportamentais,conhecimentos_tecnicos,match_competencias_v3
33971,Profissional com experiência com SAP EWM.,SAP MM/WM EWM,1.000000
48726,Experiência com SAP MM,SAP MM,1.000000
9955,SAP SD,SAP SD,1.000000
33259,"Conhecimentos:\n\nC#\n.Net framework - 3.x, 4....",.Net Framework 4.0+ Core 2.0+ MVC WebForms Sql...,0.776770
48732,Experiencia com SAP IPPE,iPPE DMS MM PP PM PS,0.738246
43757,Alocação do consultor Part Time para apoio ao ...,"Abap, PA, PY, Success Factory",0.732079
6218,Desejável:\n\nAcademia SAP ABAP\nCertificação ...,SAP ABAP e SAP PI/PO,0.705444
1691,Desejável:\n\nAcademia SAP ABAP\nCertificação ...,SAP ABAP e SAP PI/PO,0.705444
52242,Experiência com CRM Siebel para atuar com supo...,Siebel,0.703972
25282,Consultor/Desenvolvedor - SAP Lumira.... esta ...,SAP BW/BO/LUMIRA/DASHBOARD/ SAP SD,0.635011


### Match entre título da vaga e título profissional do candidato


Para capturar a aderência do título profissional do candidato em relação ao título da vaga, foi utilizada a mesma abordagem de TF-IDF + Similaridade do Cosseno. Foram concatenados os textos de titulo_vaga e perfil_candidato (formado por titulo_profissional e, em alguns casos, o objetivo). Assim, foi possível gerar um score de similaridade textual salvo em match_titulo_vaga_perfil_v2. Isso ajuda a identificar quando, por exemplo, a vaga é para "Consultor SAP SD" e o candidato se apresenta com título similar.



In [ ]:
df_final['perfil_candidato'] = (
    df_final['titulo_profissional'].fillna('') + ' ' +
    df_final['objetivo_profissional'].fillna('')
).str.lower().str.strip()

def calcular_match_textual(texto1, texto2):
    if pd.isna(texto1) or pd.isna(texto2):
        return 0
    set1 = set(texto1.lower().strip().split())
    set2 = set(texto2.lower().strip().split())
    intersecao = set1 & set2
    return len(intersecao) / len(set1) if set1 else 0

df_final['match_titulo_vaga_perfil'] = df_final.apply(
    lambda row: calcular_match_textual(row['titulo_vaga'], row['perfil_candidato']), axis=1
)


In [ ]:
df_final.sort_values(by='match_titulo_vaga_perfil', ascending=False)[[
    'perfil_candidato',
    'titulo_vaga',
    'match_titulo_vaga_perfil'
]].head(10)

,perfil_candidato,titulo_vaga,match_titulo_vaga_perfil
56406,consultor especialista em mainframe consultor ...,Mainframe,1.0
32412,delivery manager / project manager delivery ma...,Delivery Manager,1.0
19146,consultor sap mm/wm sr. consultor sap mm/wm sr.,SAP MM/WM,1.0
32596,consultor técnico peoplesoft consultor técnico...,Consultor Peoplesoft,1.0
32595,consultor técnico peoplesoft at resource it s ...,Consultor Peoplesoft,1.0
32943,consultor sap sd consultor sap sd,Consultor SAP SD,1.0
9476,scrum master / project manager scrum master / ...,Scrum Master,1.0
9370,administrador de redes administrador de redes,Administrador de Redes,1.0
32860,analista desenvolvedor java analista desenvolv...,Java,1.0
10367,uma vaga para consultor sap ewm uma vaga para ...,SAP EWM,1.0


In [ ]:
df_final['perfil_candidato'] = (
    df_final['titulo_profissional'].fillna('') + ' ' + df_final['objetivo_profissional'].fillna('')
)

corpus_titulo = df_final['titulo_vaga'].fillna('') + ' ||| ' + df_final['perfil_candidato'].fillna('')

vagas_text = corpus_titulo.apply(lambda x: x.split(' ||| ')[0])
perfis_text = corpus_titulo.apply(lambda x: x.split(' ||| ')[1])


In [ ]:
nltk.download('stopwords')
stopwords_pt = stopwords.words('portuguese')

# Vetorizador
vectorizer = TfidfVectorizer(stop_words=stopwords_pt)

# Aplica nos textos concatenados
tfidf_matrix = vectorizer.fit_transform(vagas_text.tolist() + perfis_text.tolist())

n = len(df_final)
vagas_tfidf = tfidf_matrix[:n]
perfis_tfidf = tfidf_matrix[n:]

# Similaridade de cosseno entre pares (vaga[i], perfil[i])
sim_titulo_perfil = cosine_similarity(vagas_tfidf, perfis_tfidf)

# Extrai a diagonal — correspondência linha a linha
df_final['match_titulo_vaga_perfil_v2'] = np.diag(sim_titulo_perfil)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
df_final.sort_values(by='match_titulo_vaga_perfil_v2', ascending=False)[
    ['titulo_vaga', 'perfil_candidato', 'match_titulo_vaga_perfil_v2']
].head(10)


,titulo_vaga,perfil_candidato,match_titulo_vaga_perfil_v2
28665,Desenvolvedor Full Stack JAVA,Desenvolvedor FULL STACK Java Desenvolvedor FU...,1.0
33878,Analista de Testes,Analista de Testes Analista de Testes,1.0
27837,Analista de Automação de Testes (I),Analista de Automação de Testes. Analista de A...,1.0
27841,Analista de Automação de Testes (I),Analista de Testes Automação Analista de Teste...,1.0
4103,SAP MM WM,SAP MM WM SAP MM WM,1.0
34105,SAP ABAP,SAP ABAP SAP ABAP,1.0
50399,Desenvolvedor Front End,Desenvolvedor Front-End Desenvolvedor Front-End,1.0
33874,Analista de Testes,Analista de Testes Analista de Testes,1.0
27745,TEAM LÍDER COBOL,Team Líder Cobol Team Líder Cobol,1.0
49798,Analista Sistemas,Analista de Sistemas Analista de Sistemas,1.0


### Área de atuação

A coluna area_atuacao continha descrições altamente variadas, com muitos valores únicos, múltiplas áreas separadas por vírgulas e inconsistências de formatação (como uso de maiúsculas/minúsculas, duplicações e nomes ambíguos). Para torná-la útil como variável categórica no modelo, realizamos um processo de padronização e agrupamento semântico.

Primeiro, aplicamos uma limpeza textual (remoção de espaços, padronização de acentuação e letras minúsculas). Depois, extraímos as áreas individuais de cada linha e mapeamos para grupos temáticos mais amplos, como:

desenvolvimento, infraestrutura, sap, dados, etc.

Esse agrupamento foi feito com base em palavras-chave encontradas na lista e permitiu reduzir centenas de combinações para cerca de 10 categorias generalizadas. Em seguida, a nova coluna area_atuacao_grupo foi usada para geração de variáveis dummies (area_atuacao_grupo_*) para alimentar o modelo com informações mais estruturadas e reduzidas em dimensionalidade.

In [ ]:
df_final['area_atuacao'].value_counts(dropna=False)


,count
area_atuacao,
TI - Desenvolvimento/Programação,1363
TI - SAP,958
,707
TI - Projetos,387
TI - Infraestrutura,258
...,...
"Administrativa, Gestão e Alocação de Recursos de TI, Qualidade Corporativa, Relacionamento Técnico",1
"Administrativa, Comercial, Financeira/Controladoria, Gestão e Alocação de Recursos de TI, Jurídica, Marketing, Novos Negócios e Parcerias, Qualidade Corporativa, Recursos Humanos, Relacionamento Técnico, TI - Arquitetura, TI - Banco de Dados, TI - Desenvolvimento/Design, TI - Desenvolvimento/Mobile, TI - Desenvolvimento/Programação, TI - Governança, TI - Infraestrutura, TI - Processos e Negócios, TI - Projetos, TI - Qualidade/Testes, TI - SAP, TI - Sistemas e Ferramentas, TI - Suporte, TI - Telecom, TI - UX",1
"TI - Arquitetura, TI - Governança, TI - Infraestrutura, TI - Processos e Negócios, TI - Sistemas e Ferramentas, TI - Suporte",1


In [ ]:
#df_final['area_atuacao'].dropna().unique().tolist()


In [ ]:
def agrupar_area(texto):
    if pd.isna(texto) or texto.strip() == '' or texto.lower().strip() in ['não informado']:
        return 'nao_informado'

    texto = texto.lower()

    if 'desenvolvimento' in texto or 'programa' in texto or 'design' in texto or 'mobile' in texto or 'arquitetura' in texto:
        return 'desenvolvimento'
    if 'infraestrutura' in texto or 'suporte' in texto or 'sistemas e ferramentas' in texto or 'telecom' in texto:
        return 'infraestrutura'
    if 'dados' in texto or 'banco de dados' in texto or 'bi' in texto:
        return 'dados'
    if 'projeto' in texto:
        return 'projetos'
    if 'governança' in texto or 'gestão' in texto:
        return 'governanca'
    if 'qualidade' in texto or 'teste' in texto:
        return 'qualidade'
    if 'sap' in texto:
        return 'sap'
    if 'segurança' in texto:
        return 'seguranca'
    if 'ux' in texto:
        return 'ux'
    if any(x in texto for x in ['administrativa', 'comercial', 'financeira', 'jurídica', 'controladoria']):
        return 'negocio/adm'
    if 'relacionamento técnico' in texto:
        return 'relacionamento'
    return 'outros'


In [ ]:
df_final['area_atuacao_grupo'] = df_final['area_atuacao'].apply(agrupar_area)

In [ ]:
df_final['area_atuacao_grupo'].value_counts(normalize=True, dropna=False)

,proportion
area_atuacao_grupo,
desenvolvimento,0.373494
sap,0.179796
nao_informado,0.131047
infraestrutura,0.121779
projetos,0.091937
outros,0.029842
dados,0.025209
qualidade,0.022428
governanca,0.014458


# Aplicação dos modelos

In [ ]:
colunas_modelo = [
    # Vaga
    'vaga_id',
    'area_atuacao_grupo',

    # Candidato
    'codigo_candidato',

    # Prospect
    'foi_contratado_encaminhado',

    # Novos campos objetivos
    'delta_academico',
    'delta_ingles',
    'delta_senioridade',
    'match_senioridade_aceitavel',
    'match_competencias_v3',
    'match_titulo_vaga_perfil_v2',
    'match_local'
]


df_filtrado = df_final[colunas_modelo].copy()

In [ ]:
# Colunas categóricas que queremos codificar
colunas_categoricas = [
    'area_atuacao_grupo'
]


# Preencher nulos
df_filtrado[colunas_categoricas] = df_filtrado[colunas_categoricas].fillna('nao informado')

# Aplicar get_dummies para transformar as categorias em colunas binárias
df_encoded = pd.get_dummies(df_filtrado[colunas_categoricas], drop_first=True)

# Concatenar com as colunas numéricas e o target
df_modelo_codificado = pd.concat([
    df_encoded,
    df_filtrado[['delta_academico',
    'delta_ingles',
    'delta_senioridade',
    'match_senioridade_aceitavel',
    'match_competencias_v3'  ,
    'match_titulo_vaga_perfil_v2',
    'match_local',
    'foi_contratado_encaminhado']]
], axis=1)


In [ ]:
df_modelo_codificado.head()

,area_atuacao_grupo_desenvolvimento,area_atuacao_grupo_governanca,area_atuacao_grupo_infraestrutura,area_atuacao_grupo_nao_informado,area_atuacao_grupo_negocio/adm,area_atuacao_grupo_outros,area_atuacao_grupo_projetos,area_atuacao_grupo_qualidade,area_atuacao_grupo_sap,area_atuacao_grupo_seguranca,area_atuacao_grupo_ux,delta_academico,delta_ingles,delta_senioridade,match_senioridade_aceitavel,match_competencias_v3,match_titulo_vaga_perfil_v2,match_local,foi_contratado_encaminhado
2,False,False,False,False,False,False,True,False,False,False,False,6,4,-14,0,0.177653,0.000000,1,1
26,False,False,True,False,False,False,False,False,False,False,False,-12,4,-8,0,0.073648,0.000000,0,1
28,False,False,False,False,False,False,True,False,False,False,False,0,1,-8,0,0.078225,0.000000,0,0
33,False,False,False,False,False,False,True,False,False,False,False,-12,1,-8,0,0.009540,0.351084,0,1
56,True,False,False,False,False,False,False,False,False,False,False,6,2,-14,0,0.176109,0.317858,1,0


In [ ]:
from sklearn.model_selection import train_test_split

# Separar features e target
X = df_modelo_codificado.drop(columns=['foi_contratado_encaminhado'])
y = df_modelo_codificado['foi_contratado_encaminhado']

# Treinar/testar com stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

In [ ]:
features = X_train.columns.to_list()
features


['area_atuacao_grupo_desenvolvimento',
 'area_atuacao_grupo_governanca',
 'area_atuacao_grupo_infraestrutura',
 'area_atuacao_grupo_nao_informado',
 'area_atuacao_grupo_negocio/adm',
 'area_atuacao_grupo_outros',
 'area_atuacao_grupo_projetos',
 'area_atuacao_grupo_qualidade',
 'area_atuacao_grupo_sap',
 'area_atuacao_grupo_seguranca',
 'area_atuacao_grupo_ux',
 'delta_academico',
 'delta_ingles',
 'delta_senioridade',
 'match_senioridade_aceitavel',
 'match_competencias_v3',
 'match_titulo_vaga_perfil_v2',
 'match_local']

### Regressão Logística

A Regressão Logística foi incluída como modelo base para comparação por ser um método estatístico clássico, simples de interpretar e rápido de treinar. Apesar de menos complexo, ela é útil como benchmark para sabermos o quanto os modelos mais sofisticados estão realmente entregando ganhos adicionais. Embora tenha apresentado desempenho inferior neste caso, sua inclusão foi fundamental para validar a necessidade de modelos mais avançados.



In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix

modelo_lr = LogisticRegression(
    class_weight='balanced',
    max_iter=1000,
    random_state=42
)
modelo_lr.fit(X_train, y_train)

y_pred = modelo_lr.predict(X_test)
print(classification_report(y_test, y_pred))



              precision    recall  f1-score   support

           0       0.68      0.67      0.68      1027
           1       0.44      0.45      0.45       592

    accuracy                           0.59      1619
   macro avg       0.56      0.56      0.56      1619
weighted avg       0.59      0.59      0.59      1619



### Random Forest

Optou-se por utilizar o Random Forest como um dos modelos principais por ser um algoritmo robusto, interpretável e eficaz em lidar com dados tabulares heterogêneos, como os que temos neste projeto (variáveis numéricas, categóricas, textuais transformadas). Ele funciona como um conjunto de árvores de decisão que votam para tomar uma decisão final, o que o torna menos sensível a overfitting do que uma árvore única. Além disso, o Random Forest nos fornece importâncias de variáveis (feature importances) que são muito úteis para entender quais fatores mais influenciam a contratação ou o encaminhamento dos candidatos. É um ótimo baseline para problemas de classificação binária como o nosso.



In [ ]:
from sklearn.ensemble import RandomForestClassifier


# Criar o modelo com class_weight balanceado
modelo = RandomForestClassifier(class_weight='balanced', random_state=42)
modelo.fit(X_train, y_train)

# Avaliação
y_pred = modelo.predict(X_test)
print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.66      0.79      0.72      1027
           1       0.46      0.31      0.37       592

    accuracy                           0.61      1619
   macro avg       0.56      0.55      0.55      1619
weighted avg       0.59      0.61      0.59      1619

[[811 216]
 [409 183]]


In [ ]:
y.value_counts(normalize=True)


,proportion
foi_contratado_encaminhado,
0,0.634106
1,0.365894


### XG Boost

Foi escolhido também o XGBoost (Extreme Gradient Boosting) por ser um dos algoritmos mais poderosos e competitivos em tarefas de classificação tabular. Ele constrói árvores de decisão de forma sequencial, otimizando o erro cometido pelas anteriores, o que resulta em um modelo altamente preciso e ajustável via hyperparâmetros. XGBoost tem excelente desempenho em datasets desbalanceados como o nosso, onde o número de contratados/encaminhados é bem menor do que o de descartados, e permite usar técnicas como ponderação por classe e ajuste de limiar de decisão. No nosso caso, o XGBoost mostrou maior capacidade de recuperar bons candidatos, mesmo quando o Random Forest mantinha uma alta acurácia sem encontrar tantos positivos.

In [ ]:
from xgboost import XGBClassifier

modelo_xgb = XGBClassifier(
    scale_pos_weight=(len(y) - y.sum()) / y.sum(),  # lida com desbalanceamento
    use_label_encoder=False,
    eval_metric='logloss',
    random_state=42
)
modelo_xgb.fit(X_train, y_train)


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [03:01:13] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=None, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='logloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None,
              learning_rate=None, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=None,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, num_parallel_tree=None, random_state=42, ...)

In [ ]:
y_pred = modelo_xgb.predict(X_test)
print(classification_report(y_test, y_pred))


              precision    recall  f1-score   support

           0       0.67      0.68      0.68      1027
           1       0.44      0.43      0.43       592

    accuracy                           0.59      1619
   macro avg       0.56      0.55      0.56      1619
weighted avg       0.59      0.59      0.59      1619



Foi testado antes somente com os contratados como alvo, resultando em uma acurácia de 94%, mas precision e recall em torno de 11-20%. Antes, a classe positiva era só os contratados (~1% dos dados), e o modelo podia acertar quase tudo dizendo "não vai ser contratado" —> alta acurácia, mas baixo recall para 1.
Agora, com a nova definição a classe positiva (contratado ou encaminhado) está bem mais equilibrada (593 positivos de 1619 exemplos).

O modelo é forçado a arriscar mais, e por isso erra mais no geral —> acurácia cai, mas:
Recall e precision da classe 1 sobem muito, o que é o foco aqui.
Passamos a identificar mais perfis relevantes de verdade.

In [ ]:
y_proba = modelo_xgb.predict_proba(X_test)[:, 1]

In [ ]:
thresholds = np.arange(0.1, 0.9, 0.1)
for t in thresholds:
    y_pred_t = (y_proba > t).astype(int)
    print(f"Threshold: {t}")
    print(classification_report(y_test, y_pred_t, digits=3))
    print("-" * 50)


Threshold: 0.1
              precision    recall  f1-score   support

           0      0.788     0.040     0.076      1027
           1      0.371     0.981     0.538       592

    accuracy                          0.384      1619
   macro avg      0.580     0.511     0.307      1619
weighted avg      0.636     0.384     0.245      1619

--------------------------------------------------
Threshold: 0.2
              precision    recall  f1-score   support

           0      0.769     0.146     0.245      1027
           1      0.384     0.924     0.543       592

    accuracy                          0.431      1619
   macro avg      0.577     0.535     0.394      1619
weighted avg      0.628     0.431     0.354      1619

--------------------------------------------------
Threshold: 0.30000000000000004
              precision    recall  f1-score   support

           0      0.712     0.298     0.420      1027
           1      0.394     0.791     0.526       592

    accuracy       

Como o sistema serve para ajudar o RH a não perder candidatos bons (encaminhar mais corretamente), a melhor escolha parece ser:

Threshold: 0.3

Ele recupera cerca de 80% dos bons candidatos, errando moderadamente,uma troca justa para apoio à decisão, onde o RH ainda pode fazer a triagem final.



# Deploy no Streamlit

In [ ]:
# Salvar modelo puro
import joblib

joblib.dump(modelo_xgb, 'modelo_xgb_final.pkl')


['modelo_xgb_final.pkl']

### Dataframe de vagas tratado

In [ ]:
colunas_candidato = [
    # Candidato
    'codigo_candidato',
    'objetivo_profissional',
    'nome',
    'local',
    'titulo_profissional',
    'area_atuacao',
    'conhecimentos_tecnicos',
    'nivel_profissional',
    'nivel_academico_x',
    'nivel_ingles_x',
    'area_atuacao_grupo'
]
colunas_atuacao = [
    'area_atuacao_grupo_desenvolvimento',
    'area_atuacao_grupo_governanca',
    'area_atuacao_grupo_infraestrutura',
    'area_atuacao_grupo_nao_informado',
    'area_atuacao_grupo_negocio/adm',
    'area_atuacao_grupo_outros',
    'area_atuacao_grupo_projetos',
    'area_atuacao_grupo_qualidade',
    'area_atuacao_grupo_sap',
    'area_atuacao_grupo_seguranca',
    'area_atuacao_grupo_ux'
]

df_candidatos_tratado = df_final[colunas_candidato].copy()
df_candidatos_areas = df_modelo_codificado[colunas_atuacao].copy()





In [ ]:
assert all(df_candidatos_tratado.index == df_candidatos_areas.index), "Índices desalinhados"

In [ ]:
df_candidatos_streamlit = pd.concat(
    [df_candidatos_tratado.set_index('codigo_candidato'),
     df_candidatos_areas.set_index(df_candidatos_tratado['codigo_candidato'])],
    axis=1
).reset_index()


In [ ]:
df_candidatos_streamlit = df_candidatos_streamlit.drop_duplicates(subset='codigo_candidato')

In [ ]:
df_candidatos_streamlit.to_csv("df_candidatos_tratado.csv", index=False)

In [ ]:
import joblib

colunas_modelo = X.columns.tolist()

# Salvar
joblib.dump(colunas_modelo, 'colunas_modelo.pkl')


['colunas_modelo.pkl']